In [1]:
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns; sns.set()
%matplotlib inline

import keras
from keras import objectives
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Multiply, Add
from keras.optimizers import Adam, Nadam
import salty
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from random import shuffle
import pandas as pd
import random

#Keras build
from keras import backend as K
from keras.objectives import binary_crossentropy #objs or losses
from keras.models import Model
from keras.layers import Input, Dense, Lambda, Layer
from keras.layers.core import Dense, Activation, Flatten, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import GRU
from keras.layers.convolutional import Convolution1D

#cation data
cations = pd.read_csv('../data/cations.csv')
cations = cations['smiles_string']
salts = pd.read_csv('../data/salts.csv')
salts = salts['smiles_string']
categories = pd.read_csv('../data/categories.csv')
categories = categories['category']
coldic = pd.read_csv('../data/coldic.csv')
coldic = coldic.to_dict(orient='records')[0]
salt_coldic = pd.read_csv('../data/salt_coldic.csv')
salt_coldic = salt_coldic.to_dict(orient='records')[0]
salt_categories = pd.read_csv('../data/salt_categories.csv')
salt_categories = salt_categories['category']
density_coldic = pd.read_csv('../data/density_coldic.csv')
density_coldic = density_coldic.to_dict(orient='records')[0]
density_categories = pd.read_csv('../data/density_categories.csv')
density_categories = density_categories['category']

#supporting functions
import sys
sys.path.insert(0, '../')
from scripts import *

#training array info
smile_max_length = 105
import json
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

/home/wesleybeckner/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
#plot
import matplotlib.pylab as plt
import seaborn as sns; sns.set()
from matplotlib import colors
from itertools import cycle

#data
import pandas as pd
import numpy as np
from numpy.linalg import norm
import json
import random
import copy

#ML
import keras
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import backend as K
from keras import objectives
from keras.objectives import binary_crossentropy #objs or losses
from keras.layers import Dense, Dropout, Input, Multiply, Add, Lambda
from keras.layers.core import Dense, Activation, Flatten, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import GRU
from keras.layers.convolutional import Convolution1D

#chem
import salty
from rdkit import Chem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import Draw


class MoleculeVAE():
    

    autoencoder = None
    
    def create(self,
               charset,
               max_length = 105,
               latent_rep_size = 292,
               weights_file = None,
               qspr = False,
               conv_layers=3,
               gru_layers=3,
               conv_filters=3, 
               conv_kernel_size=9,
               gru_output_units=501):
        charset_length = len(charset)
    
        x = Input(shape=(max_length, charset_length))
        _, z = self._buildEncoder(x, 
                                  latent_rep_size, 
                                  max_length,
                                  conv_layers, 
                                  conv_filters, 
                                  conv_kernel_size)
        self.encoder = Model(x, z)
        encoded_input = Input(shape=(latent_rep_size,))
        self.decoder = Model(
            encoded_input,
            self._buildDecoder(
                encoded_input,
                latent_rep_size,
                max_length,
                charset_length,
                gru_layers,
                gru_output_units
            )
        )

        x1 = Input(shape=(max_length, charset_length))
        vae_loss, z1 = self._buildEncoder(x1, 
                                          latent_rep_size, 
                                          max_length,
                                          conv_layers, 
                                          conv_filters,
                                          conv_kernel_size)
        
        if qspr:
            self.autoencoder = Model(
                x1,
                self._buildDecoderQSPR(
                    z1,
                    latent_rep_size,
                    max_length,
                    charset_length,
                    gru_layers,
                    gru_output_units
                )
            )

        else:
            self.autoencoder = Model(
                x1,
                self._buildDecoder(
                    z1,
                    latent_rep_size,
                    max_length,
                    charset_length,
                    gru_layers,
                    gru_output_units
                )
            )
            
        self.qspr = Model(
            x1,
            self._buildQSPR(
                z1,
                latent_rep_size,
                max_length,
                charset_length
            )
        )


        if weights_file:
            self.autoencoder.load_weights(weights_file, by_name = True)
            self.encoder.load_weights(weights_file, by_name = True)
            self.decoder.load_weights(weights_file, by_name = True)
            self.qspr.load_weights(weights_file, by_name = True)
        if qspr:
            self.autoencoder.compile(optimizer = 'Adam',
                                     loss = {'decoded_mean': vae_loss, 'qspr': 'mean_squared_error'},
                                     metrics = ['accuracy', 'mse'])
        else:
            self.autoencoder.compile(optimizer = 'Adam',
                                     loss = {'decoded_mean': vae_loss},
                                     metrics = ['accuracy'])
    def _buildEncoder(self, x, latent_rep_size, max_length, epsilon_std = 0.01, conv_layers=3, conv_filters=9, conv_kernel_size=9):
        h = Convolution1D(9, 9, activation = 'relu', name='conv_1')(x)
        for convolution in range(conv_layers-2):
            h = Convolution1D(conv_filters, conv_kernel_size, activation = 'relu', name='conv_{}'.format(convolution+2))(h)
        h = Convolution1D(10, 11, activation = 'relu', name='conv_{}'.format(conv_layers))(h)
        h = Flatten(name='flatten_1')(h)
        h = Dense(435, activation = 'relu', name='dense_1')(h)

        def sampling(args):
            z_mean_, z_log_var_ = args
            batch_size = K.shape(z_mean_)[0]
            epsilon = K.random_normal(shape=(batch_size, latent_rep_size), mean=0., stddev = epsilon_std)
            return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

        z_mean = Dense(latent_rep_size, name='z_mean', activation = 'linear')(h)
        z_log_var = Dense(latent_rep_size, name='z_log_var', activation = 'linear')(h)

        def vae_loss(x, x_decoded_mean):
            x = K.flatten(x)
            x_decoded_mean = K.flatten(x_decoded_mean)
            xent_loss = max_length * objectives.binary_crossentropy(x, x_decoded_mean)
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = -1)
            return xent_loss + kl_loss

        return (vae_loss, Lambda(sampling, output_shape=(latent_rep_size,), name='lambda')([z_mean, z_log_var]))

    def _buildDecoderQSPR(self, z, latent_rep_size, max_length, charset_length, gru_layers=3):

        h = Dense(latent_rep_size, name='latent_input', activation = 'relu')(z)
        h = RepeatVector(max_length, name='repeat_vector')(h)
        h = GRU(501, return_sequences = True, name='gru_1')(h)
        for gru in range(gru_layers-2):
            h = GRU(501, return_sequences = True, name='gru_{}'.format(gru+2))(h)
        h = GRU(501, return_sequences = True, name='gru_{}'.format(gru_layers))(h)
        smiles_decoded = TimeDistributed(Dense(charset_length, activation='softmax'), name='decoded_mean')(h)

        h = Dense(latent_rep_size, name='qspr_input', activation='relu')(z)
        h = Dense(100, activation='relu', name='hl_1')(h)
        h = Dropout(0.5)(h)
        smiles_qspr = Dense(1, activation='linear', name='qspr')(h)

        return smiles_decoded, smiles_qspr

    def _buildDecoder(self, z, latent_rep_size, max_length, charset_length, gru_layers=3, gru_output_units=501):

        h = Dense(latent_rep_size, name='latent_input', activation = 'relu')(z)
        h = RepeatVector(max_length, name='repeat_vector')(h)
        h = GRU(501, return_sequences = True, name='gru_1')(h)
        for gru in range(gru_layers-2):
            h = GRU(gru_output_units, return_sequences = True, name='gru_{}'.format(gru+2))(h)
        h = GRU(501, return_sequences = True, name='gru_{}'.format(gru_layers))(h)
        smiles_decoded = TimeDistributed(Dense(charset_length, activation='softmax'), name='decoded_mean')(h)

        return smiles_decoded
    
    def _buildQSPR(self, z, latent_rep_size, max_length, charset_length):
        h = Dense(latent_rep_size, name='latent_input', activation='relu')(z)
        h = Dense(100, activation='relu', name='hl_1')(h)
        h = Dropout(0.5)(h)
        return Dense(1, activation='linear', name='qspr')(h)

    def save(self, filename):
        self.autoencoder.save_weights(filename)
    
    def load(self, charset, weights_file, latent_rep_size = 292):
        self.create(charset, weights_file = weights_file, latent_rep_size = latent_rep_size)

In [2]:
vae = MoleculeVAE()
vae.create?

Signature: vae.create(charset, max_length=105, latent_rep_size=292, weights_file=None, qspr=False)
Docstring: <no docstring>
File:      /media/wesleybeckner/weshhd/wes/Dropbox/Python/py3/generative_learners_salts/scripts/vae.py
Type:      method


In [6]:
#supporting functions
import sys
sys.path.insert(0, '../')
from scripts import *

In [10]:
f = open("../data/gdb_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

In [15]:
vae.create(char_set, 
           max_length=51, 
           latent_rep_size=292,
           weights_file='../models/1mil_GDB17_10.h5',
           qspr=False)

In [16]:
#new model
vae.autoencoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 51, 33)       0                                            
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 43, 9)        2682        input_15[0][0]                   
__________________________________________________________________________________________________
conv_2 (Conv1D)                 (None, 35, 9)        738         conv_1[0][0]                     
__________________________________________________________________________________________________
conv_3 (Conv1D)                 (None, 25, 10)       1000        conv_2[0][0]                     
__________________________________________________________________________________________________
flatten_1 

In [12]:
#original model
vae.autoencoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 51, 33)       0                                            
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 43, 9)        2682        input_12[0][0]                   
__________________________________________________________________________________________________
conv_2 (Conv1D)                 (None, 35, 9)        738         conv_1[0][0]                     
__________________________________________________________________________________________________
conv_3 (Conv1D)                 (None, 25, 10)       1000        conv_2[0][0]                     
__________________________________________________________________________________________________
flatten_1 

In [18]:
anions = []
for smi in salts:
    anions.append(smi.split('.')[1])

In [24]:
lens = []
for anion in anions:
    lens.append(len(anion))

In [25]:
max(lens)

62

In [39]:
#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=62, conv_layers=3, conv_filters=3, conv_kernel_size=9)

In [40]:
chemvae.autoencoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_47 (InputLayer)           (None, 62, 37)       0                                            
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 54, 9)        3006        input_47[0][0]                   
__________________________________________________________________________________________________
conv_2 (Conv1D)                 (None, 46, 9)        738         conv_1[0][0]                     
__________________________________________________________________________________________________
conv_3 (Conv1D)                 (None, 36, 10)       1000        conv_2[0][0]                     
__________________________________________________________________________________________________
flatten_1 

In [42]:
import json
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 62
df = pd.read_csv('../data/GDB/GDB17.1000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=62)
data_size = 100000
histories = []
for p in range(10):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    #for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
    for i, smile in enumerate(padded_smiles[:data_size]):
#         linearly_scaled_prob = random.random() < 0.5#i/data_size
#         if linearly_scaled_prob:
#             smile = random.choice(cations)
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1

    X_train, X_test = train_test_split(X_train, test_size=0.01, random_state=42)   
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False, validation_data=(X_test, X_test))
    histories.append(history.history)
    chemvae.save('../models/1mil_GDB17_62smi_{}.h5'.format(p+1))
    
with open('../models/history_{}.json'.format('1mil_GDB17_62smi'), 'w') as f:
        json.dump(histories, f)

Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1768s 18ms/step - loss: 2.3151 - acc: 0.7021 - val_loss: 2.2591 - val_acc: 0.7072
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1768s 18ms/step - loss: 2.2513 - acc: 0.7070 - val_loss: 2.2269 - val_acc: 0.7091
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1766s 18ms/step - loss: 2.3051 - acc: 0.7029 - val_loss: 2.2560 - val_acc: 0.7078
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1769s 18ms/step - loss: 2.2421 - acc: 0.7072 - val_loss: 2.2396 - val_acc: 0.7079
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1790s 18ms/step - loss: 2.2341 - acc: 0.7077 - val_loss: 2.2339 - val_acc: 0.7085
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
9

In [43]:
import json
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 51
df = pd.read_csv('../data/GDB/GDB17.1000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=51)
data_size = 100000
histories = []
for p in range(10):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    #for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
    for i, smile in enumerate(padded_smiles[:data_size]):
#         linearly_scaled_prob = random.random() < 0.5#i/data_size
#         if linearly_scaled_prob:
#             smile = random.choice(cations)
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1

    X_train, X_test = train_test_split(X_train, test_size=0.01, random_state=42)   
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False, validation_data=(X_test, X_test))
    histories.append(history.history)
    chemvae.save('../models/1mil_GDB17_51smi_{}.h5'.format(p+1))
    
with open('../models/history_{}.json'.format('1mil_GDB17_51smi'), 'w') as f:
        json.dump(histories, f)

Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1452s 15ms/step - loss: 2.2725 - acc: 0.6401 - val_loss: 2.1741 - val_acc: 0.6477
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1455s 15ms/step - loss: 2.1652 - acc: 0.6484 - val_loss: 2.1403 - val_acc: 0.6513
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1465s 15ms/step - loss: 2.1700 - acc: 0.6480 - val_loss: 2.2017 - val_acc: 0.6466
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1471s 15ms/step - loss: 2.1640 - acc: 0.6482 - val_loss: 2.1584 - val_acc: 0.6497
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1483s 15ms/step - loss: 2.1702 - acc: 0.6478 - val_loss: 2.1526 - val_acc: 0.6515
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
9

In [44]:
import json
f = open("../data/gdb_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 51
df = pd.read_csv('../data/GDB/GDB17.1000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=51)
data_size = 100000
histories = []
for p in range(10):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    #for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
    for i, smile in enumerate(padded_smiles[:data_size]):
#         linearly_scaled_prob = random.random() < 0.5#i/data_size
#         if linearly_scaled_prob:
#             smile = random.choice(cations)
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1

    X_train, X_test = train_test_split(X_train, test_size=0.01, random_state=42)   
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False, validation_data=(X_test, X_test))
    histories.append(history.history)
    chemvae.save('../models/1mil_GDB17_51smi_gdbchar_{}.h5'.format(p+1))
    
with open('../models/history_{}.json'.format('1mil_GDB17_51smi_gdbchar'), 'w') as f:
        json.dump(histories, f)

Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1448s 15ms/step - loss: 2.5882 - acc: 0.6382 - val_loss: 2.5091 - val_acc: 0.6444
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1451s 15ms/step - loss: 2.5242 - acc: 0.6436 - val_loss: 2.4950 - val_acc: 0.6455
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1459s 15ms/step - loss: 2.5310 - acc: 0.6432 - val_loss: 2.5334 - val_acc: 0.6438
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1466s 15ms/step - loss: 2.5332 - acc: 0.6429 - val_loss: 2.5431 - val_acc: 0.6436
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1480s 15ms/step - loss: 2.5152 - acc: 0.6440 - val_loss: 2.5146 - val_acc: 0.6455
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
9

In [45]:
import json
f = open("../data/gdb_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 51
df = pd.read_csv('../data/GDB/GDB17.1000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=51)
data_size = 100000
histories = []
for p in range(10):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    
    #for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
    for i, smile in enumerate(padded_smiles[:data_size]):
#         linearly_scaled_prob = random.random() < 0.5#i/data_size
#         if linearly_scaled_prob:
#             smile = random.choice(cations)
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1

#     X_train, X_test = train_test_split(X_train, test_size=0.01, random_state=42)   
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False)
    histories.append(history.history)
#     chemvae.save('../models/1mil_GDB17_51smi_gdbchar_{}.h5'.format(p+1))
    
# with open('../models/history_{}.json'.format('1mil_GDB17_51smi_gdbchar'), 'w') as f:
#         json.dump(histories, f)

Epoch 1/1
100000/100000 [==============================] - 1496s 15ms/step - loss: 2.5403 - acc: 0.6406
Epoch 1/1
100000/100000 [==============================] - 1493s 15ms/step - loss: 2.8774 - acc: 0.5968
Epoch 1/1
 10272/100000 [==>...........................] - ETA: 22:34 - loss: 3.9851 - acc: 0.4630

KeyboardInterrupt: 

In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[]

In [ ]:
import json
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 62
df = pd.read_csv('../data/GDB/GDB17.1000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=smile_max_length)
data_size = 100000
histories = []
for p in range(10):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    for i, smile in enumerate(padded_smiles): #was it the data_size cap?
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1
            
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False)
    histories.append(history.history)
#     chemvae.save('../models/1mil_GDB17_51smi_gdbchar_{}.h5'.format(p+1))
    
# with open('../models/history_{}.json'.format('1mil_GDB17_51smi_gdbchar'), 'w') as f:
#         json.dump(histories, f)

Epoch 1/1
 66848/100000 [===================>..........] - ETA: 10:07 - loss: 2.2579 - acc: 0.7038